In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\Downloads\\Rent_Ml_flow_AWS\\Rent_Prediction_Using_ML\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Dell\\Downloads\\Rent_Ml_flow_AWS\\Rent_Prediction_Using_ML'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Adamya113/Rent_Prediction_Using_ML.mlflow"
# os.environ["MLFLOW_TRACKING_USERNAME"]="entbappy"
# os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a369aa6f9eac5b10041d5c8edbcef0"

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [14]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.NeuralNetwork
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Adamya113/Rent_Prediction_Using_ML.mlflow",  
        )
        return model_evaluation_config


In [16]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import numpy as np
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
import dagshub

In [17]:
class linearRegression(nn.Module): # all the dependencies from torch will be given to this class [parent class] # nn.Module contains all the building block of neural networks:
  def __init__(self,input_dim):
    super(linearRegression,self).__init__()  # building connection with parent and child classes
    self.fc1=nn.Linear(input_dim,64)          # hidden layer 1
    self.fc2=nn.Linear(64,32)                  # hidden layer 2
    self.fc3=nn.Linear(32,16)                   # hidden layer 3
    self.fc4=nn.Linear(16,8)                   #hidden layer 4
    self.fc5=nn.Linear(8,1)                   # last layer

  def forward(self,d):
    out=torch.relu(self.fc1(d))              # input * weights + bias for layer 1
    out=torch.relu(self.fc2(out))            # input * weights + bias for layer 2
    out=torch.relu(self.fc3(out))            # input * weights + bias for layer 3
    out=torch.relu(self.fc4(out))            # input * weights + bias for layer 4
    out=self.fc5(out)                        # input * weights + bias for last layer
    return out                               # final outcome

In [18]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        test_x_tensor = torch.tensor(test_x.values, dtype=torch.float32)
        input_dim = test_x_tensor.shape[1]
        model = linearRegression(input_dim)
        saved_data = joblib.load(self.config.model_path)
        model.load_state_dict(saved_data['model_state_dict'])

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        dagshub.init(repo_owner='Adamya113', repo_name='Rent_Prediction_Using_ML', mlflow=True)
        with mlflow.start_run():
   
            with torch.no_grad():
                model.eval()   # make model in evaluation stage
                predicted_tensor = model(test_x_tensor)
            predicted_y =  predicted_tensor.numpy()

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_y)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.pytorch.log_model(model, "model", registered_model_name="LinearRegression")
            else:
                mlflow.pytorch.log_model(model, "model")

    


In [19]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-16 23:42:19,347: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-16 23:42:19,421: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-16 23:42:19,435: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-16 23:42:19,487: INFO: common: created directory at: artifacts]
[2025-01-16 23:42:19,498: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-16 23:42:20,921: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/Adamya113/Rent_Prediction_Using_ML "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Adamya113/Rent_Prediction_Using_ML"

[2025-01-16 23:42:20,986: INFO: helpers: Initialized MLflow to track repo "Adamya113/Rent_Prediction_Using_ML"]


Repository Adamya113/Rent_Prediction_Using_ML initialized!

[2025-01-16 23:42:21,005: INFO: helpers: Repository Adamya113/Rent_Prediction_Using_ML initialized!]
[2025-01-16 23:42:21,885: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'LinearRegression' already exists. Creating a new version of this model...
2025/01/16 23:43:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LinearRegression, version 2
Created version '2' of model 'LinearRegression'.
